In [2]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1007k      0 --:--:-- --:--:-- --:--:-- 1007k


In [3]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [4]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [5]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [6]:

# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:

# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())

print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')

print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [9]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [10]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)


In [11]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0162, 0.0180, 0.0278,  ..., 0.0315, 0.0215, 0.0217],
        [0.0241, 0.0181, 0.0279,  ..., 0.0134, 0.0438, 0.0353],
        [0.0157, 0.0123, 0.0460,  ..., 0.0197, 0.0190, 0.0230],
        ...,
        [0.0465, 0.0481, 0.0016,  ..., 0.0147, 0.0234, 0.0412],
        [0.0008, 0.0048, 0.0368,  ..., 0.0376, 0.0497, 0.0227],
        [0.0259, 0.0281, 0.0006,  ..., 0.0021, 0.0133, 0.0473]])
item_factors.weight tensor([[0.0303, 0.0141, 0.0193,  ..., 0.0098, 0.0228, 0.0252],
        [0.0274, 0.0314, 0.0101,  ..., 0.0489, 0.0311, 0.0094],
        [0.0306, 0.0035, 0.0275,  ..., 0.0310, 0.0310, 0.0383],
        ...,
        [0.0111, 0.0059, 0.0120,  ..., 0.0106, 0.0212, 0.0308],
        [0.0074, 0.0291, 0.0214,  ..., 0.0037, 0.0359, 0.0352],
        [0.0488, 0.0122, 0.0415,  ..., 0.0329, 0.0065, 0.0256]])


In [12]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-12-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.072603087739896
iter #1 Loss: 4.74474812280103
iter #2 Loss: 2.472452465955376
iter #3 Loss: 1.7198116298254371
iter #4 Loss: 1.3448472567621221
iter #5 Loss: 1.1280173980796397
iter #6 Loss: 0.9911295176308772
iter #7 Loss: 0.8999327362945237
iter #8 Loss: 0.8368555240068339
iter #9 Loss: 0.7923126424175834
iter #10 Loss: 0.759301263744456
iter #11 Loss: 0.7342754815738213
iter #12 Loss: 0.7158127043982447
iter #13 Loss: 0.7018286615685763
iter #14 Loss: 0.6901466005722884
iter #15 Loss: 0.6815041706798041
iter #16 Loss: 0.674943287057925
iter #17 Loss: 0.6695848544539534
iter #18 Loss: 0.6657170245263178
iter #19 Loss: 0.6625259071210314
iter #20 Loss: 0.660587432534259
iter #21 Loss: 0.6593097001692365
iter #22 Loss: 0.6576418244294103
iter #23 Loss: 0.6565821362933532
iter #24 Loss: 0.6559488973036636
iter #25 Loss: 0.6550392945330155
iter #26 Loss: 0.6540734829938957
iter #27 Loss: 0.6533673987322047
iter #28 Loss: 0.6522040955608871
iter #29 Loss: 0.6512400357402

In [19]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        print('param_data', iw)

user_factors.weight tensor([[ 1.0931,  0.4007,  0.7556,  ...,  1.9097,  1.6027,  1.0430],
        [ 1.5487,  1.6328,  1.6115,  ...,  0.1820,  1.4985,  1.3919],
        [-2.2606,  1.1204,  1.7185,  ...,  1.2627,  2.7522,  1.0829],
        ...,
        [ 1.2139,  2.5589,  0.7703,  ..., -0.3266,  1.4638,  1.2265],
        [ 0.8747,  0.5806,  0.8548,  ...,  0.9446,  1.5522,  1.2359],
        [-0.1782,  1.4326,  1.4130,  ...,  0.8728,  1.1992,  1.4484]],
       device='cuda:0')
param_data 0
item_factors.weight tensor([[0.4082, 0.4333, 0.8305,  ..., 0.5486, 0.0910, 0.4280],
        [0.5866, 0.0319, 0.2763,  ..., 0.2404, 0.6551, 0.3697],
        [0.3325, 0.3859, 0.5484,  ..., 0.4184, 0.4510, 0.7802],
        ...,
        [0.3012, 0.3174, 0.3232,  ..., 0.3129, 0.3317, 0.3416],
        [0.3315, 0.3992, 0.3924,  ..., 0.3751, 0.4078, 0.4049],
        [0.3681, 0.4049, 0.4332,  ..., 0.4233, 0.3982, 0.4165]],
       device='cuda:0')
param_data tensor([[0.4082, 0.4333, 0.8305,  ..., 0.5486, 0.0910, 0

In [20]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()


In [21]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [16]:

from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [17]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''


for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Jurassic Park (1993)
	 Braveheart (1995)
	 Terminator 2: Judgment Day (1991)
	 Toy Story (1995)
	 Apollo 13 (1995)
	 Star Wars: Episode VI - Return of the Jedi (1983)
	 Fugitive, The (1993)
	 Batman (1989)
	 Aladdin (1992)
	 Sixth Sense, The (1999)
Cluster #1
	 GoldenEye (1995)
	 Crimson Tide (1995)
	 Natural Born Killers (1994)
	 Casino (1995)
	 Demolition Man (1993)
	 Starship Troopers (1997)
	 Mr. Holland's Opus (1995)
	 Tombstone (1993)
	 Casper (1995)
	 Judge Dredd (1995)
Cluster #2
	 Ace Ventura: Pet Detective (1994)
	 Mask, The (1994)
	 Pirates of the Caribbean: The Curse of the Black Pearl (2003)
	 Dumb & Dumber (Dumb and Dumber) (1994)
	 Austin Powers: The Spy Who Shagged Me (1999)
	 Bourne Identity, The (2002)
	 Star Trek: First Contact (1996)
	 Ace Ventura: When Nature Calls (1995)
	 Mummy, The (1999)
	 Blues Brothers, The (1980)
Cluster #3
	 Titanic (1997)
	 Home Alone (1990)
	 Ghost (1990)
	 Santa Clause, The (1994)
	 Mission: Impossible II (2000)
	 Legends of